# LR示例

以下示例仅用于说明LR的工作流程，目前仅支持单机测试。

初始化mpc设备

In [1]:
from common import MPCInitializer
import secretflow as sf
mpc_init = MPCInitializer()
company, partner, coordinator = mpc_init.company, mpc_init.partner, mpc_init.coordinator
spu = mpc_init.spu

# 设置设备
devices = {
    'spu': spu,
    'company': company,
    'partner': partner,
    'coordinator': coordinator,
    'active_party': partner # 验证集预测结果将揭露给active_party
}

heu_devices = (mpc_init.company_heu, mpc_init.partner_heu)

/home/lvx_vitae/AnonymVFL/.conda/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
/home/lvx_vitae/AnonymVFL/.conda/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2025-08-28 15:42:57,395	INFO worker.py:1841 -- Started a local Ray instance.


(SPURuntime pid=243873) 2025-08-28 15:43:01.382 [warning] [openssl_factory.cc:OpensslDrbg:83] Yacl has been configured to use Yacl's entropy source, but unable to find one. Fallback to use openssl's default entropy srouce
(SPURuntime pid=243873) 2025-08-28 15:43:01.383 [warning] [openssl_factory.cc:OpensslDrbg:83] Yacl has been configured to use Yacl's entropy source, but unable to find one. Fallback to use openssl's default entropy srouce


(HEUSkKeeper(heu_id=133641248831280, party=company) pid=243881) [2025-08-28 15:43:05.796] [info] [thread_pool.cc:30] Create a fixed thread pool with size 15
(HEUEvaluator(heu_id=133641113251712, party=company) pid=243877) [2025-08-28 15:43:13.725] [info] [thread_pool.cc:30] Create a fixed thread pool with size 15 [repeated 3x across cluster]


双方读取数据集。数据集为csv格式，应满足第一列为id或键值。其他列为数值型特征。标签列的列标题为'y'。

将键值和特征分开读取。

In [2]:
import pandas as pd
import os
import numpy as np
project_dir = os.path.dirname(os.path.abspath(''))

def read_dataset(path: str):
    data = pd.read_csv(path)
    keys = data.iloc[:, 0].astype(str).tolist()
    private_features = data.iloc[:, 1:].to_numpy(dtype=np.float32)
    header = data.columns.tolist()
    return (keys, private_features,None), header
company_data, company_header = company(read_dataset,num_returns=2)(os.path.join(project_dir, 'Datasets/data/data/breast_hetero_host.csv'))
partner_data, partner_header = partner(read_dataset,num_returns=2)(os.path.join(project_dir, 'Datasets/data/data/breast_hetero_guest.csv'))
company_header = sf.reveal(company_header)
partner_header = sf.reveal(partner_header)
#输出交集的列标题
header = company_header[1:] + partner_header[1:]
#记录标签列的索引
y_col = header.index('y')

执行PSI

求交后得到交集共享分片，分别由company和partner持有。原本company数据的特征在share的左侧，partner数据的特征在右侧。

In [ ]:
from PSI import private_set_intersection
company_share, partner_share, bucket_labels = private_set_intersection(company_data, partner_data,heu_devices)
company_share.device(np.savetxt)('../company_share.csv',company_share, delimiter=',')
partner_share.device(np.savetxt)('../partner_share.csv',partner_share, delimiter=',')

Computing masked company cipher
Computing masked partner cipher
Computing company shares


将训练集转移到对应设备

In [ ]:
from secretflow.data.ndarray import load
def split_X_y(share):
    X = np.delete(share, y_col, axis=1)
    y = share[:, y_col].reshape(-1, 1)
    return X, y

def share2spu(company_share, partner_share):
    company_share = company_share.to(spu)
    partner_share = partner_share.to(spu)
    return spu(lambda x, y : x + y)(company_share, partner_share)

X_train_company, y_train_company = company(split_X_y,num_returns=2)(company_share)
X_train_partner, y_train_partner = partner(split_X_y,num_returns=2)(partner_share)
X_train = share2spu(X_train_company, X_train_partner)
y_train = devices['active_party'](lambda x, y : x + y)(y_train_company.to(devices['active_party']), y_train_partner.to(devices['active_party']))


读取验证集

In [ ]:
def read_val_dataset(path):
    data = pd.read_csv(path)
    private_features = data.iloc[:, 1:].to_numpy(dtype=np.float32)
    return private_features
X_test_company = company(read_val_dataset)(os.path.join(project_dir, 'Datasets/data/data/breast_hetero_host_test.csv'))
X_test_partner = partner(read_val_dataset)(os.path.join(project_dir, 'Datasets/data/data/breast_hetero_guest_test.csv'))
X_test_partner, y_test = partner(lambda arr : (arr[:,:-1], arr[:,-1].reshape(-1,1)),num_returns=2)(X_test_partner)
X_test = load({company: X_test_company, partner: X_test_partner})


训练指定的轮次

In [ ]:
from LR import SSLR
import matplotlib.pyplot as plt

model = SSLR(devices, approx=False)
accs = model.fit(X_train, y_train, X_test, y_test, n_epochs=10, batch_size=1024, val_steps=10, lr=0.1)
model.save({
    'company': '../company_model',
    'partner': '../partner_model',
},ext='csv')
plt.plot(accs,label = "SSLR",color = "blue")
